In [1]:
'''dependencies'''
import pandas as pd

import re

import nltk

from nltk.corpus import stopwords

from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tree import Tree

import datetime

import ast
import collections
import matplotlib.pyplot as plt

In [ ]:
'''Cleaning dataset'''
'''Opening the excel of GTD incidents created from script 1'''
df = pd.read_csv('GTD_caseDetails2017b.csv').drop(['Unnamed: 0'],1).dropna(subset=['1_src_0'])
df.reset_index(drop=True, inplace=True)


Cleaning the dataset

In [ ]:
'''testing'''
print(df.columns)

'''cleaning'''
df.drop (['Additional Information_0', 'Additional Information_1',
       'Additional Information_2', 'Additional Information_3',
       'Additional Information_4', 'Additional Information_5',
       'Additional Information_6', 'Additional Information_7',
       'Attack Information_0', 'Attack Information_1', 'Attack Information_2',
       'Attack Information_3', 'Casualty Information_0',
       'Casualty Information_1', 'Casualty Information_2',
       'Casualty Information_3', 'Casualty Information_4',
       'Casualty Information_5', 'Casualty Information_6','Perpetrator Statistics_1', 'Target Information (more)_0',
       'Target Information (more)_1', 'Target Information (more)_2',
       'Target Information (more)_3', 'Target Information (more)_4',
       'Target Information (more)_5', 'Target Information (more)_6',
       'Target Information (more)_7', 'Target Information (more)_8',
       'Target Information (more)_9', 'Target Information (more)_10',
       'Target Information (more)_11', 'Weapon Information_0',
       'Weapon Information_1', 'Weapon Information_2', 'Weapon Information_3',
       'Weapon Information_4'],1)

Extracting named entities

In [ ]:
'''recognise the named entities in the incident summary'''
'''create a table'''

'''cleaning strange characters in incident summary'''
ner = []

for i in df['0_incident_summary_0']:
    encode = re.sub('\s+', ' ', i).strip()
    ner.append(encode)

df2 = pd.Series(ner)
df2

In [ ]:
'''function to convert text into named_entity'''
def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    if continuous_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
            continuous_chunk.append(named_entity)

    return continuous_chunk

In [ ]:
'''creating a list of the groups involved'''

grp_lst = []
lamda_df = df[['Perpetrator Group Information_0_0','Perpetrator Group Information_1_0','Perpetrator Group Information_2_0']].replace({'\(suspected\)':''}, regex =True)

for l in range(len(lamda_df)):
    lst = [i.strip() for i in lamda_df.iloc[l] if type(i) != float] #because nan is a float
    grp_lst.append(lst)
    
grp_lst

In [ ]:
'''creating a list of named entities'''

def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

named_entity_list = []
for i in range(len(df2)):
    ner = []
    text = df2[i]
    for j in grp_lst[i]:
        if j in text:
            ner.append(j)
            n_text = text.replace(j,"")
            #print(n_text)
            a = get_continuous_chunks(n_text)
            ner.append(a)
        else:
            ner.append(get_continuous_chunks(text))
    ner = list(flatten(ner))
    print(ner)
    named_entity_list.append(ner)
named_entity_list

In [ ]:
df2[0]

In [ ]:
'''convert NER list to (A OR B) by concatenating everything into one string'''
search_terms = []
for i in named_entity_list:
    search_term = ' OR '.join(i)
    search_terms.append(search_term)
search = pd.Series(search_terms)
search
gtd_id = pd.Series(df['GTD ID:'].values)
when =  pd.Series(df['When:'].values)
country = pd.Series(df['Country:'].values)
city = pd.Series(df['City:'].values)

df3 = pd.concat([search,gtd_id,when,country,city],axis=1)
df3.columns = ['search_terms','gtd_id','datetime','country','city']
df3

df3['year'] = pd.DatetimeIndex(df3['datetime']).year
df3

In [ ]:
#unhash to store files
#df3.to_csv('2017_cases.csv')

Searching for links

In [2]:
'''#2: This portion onwards is to search for links'''

import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from random import randint
import time


def search_session(search_term):
    driver = webdriver.Firefox()

    driver.implicitly_wait(30)
    driver.maximize_window()

    # navigate to the application home page
    driver.get("http://www.google.com.sg")

    # get the search textbox
    search_field = driver.find_element_by_id("lst-ib")
    search_field.clear()

    # enter search keyword and submit
    search_field.send_keys(search_term)
    rand = randint(8, 15)
    driver.implicitly_wait(rand)
    search_field.submit()
    
    links = []
    #collect three pages worth of links
    for i in range(3):
        #collect links
        driver.implicitly_wait(5)
        g_links = driver.find_elements_by_css_selector("div.g")
        for i in g_links:
            link = i.find_element_by_tag_name("a")
            href = link.get_attribute("href")
            links.append(href)
        #/html/body/div[5]/div[3]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div/div/div/div[2]/div/div[1]/div/div/h3/a
        #/html/body/div[5]/div[3]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div/div/div/div[2]/div/div[1]/div/div/h3/a
        
        #click next
        try:
            next_select = driver.find_element_by_css_selector("#pnnext")
            next_select.click()
        except:
            break
    
    driver.quit()
    
    return links

In [3]:
'''load the dataframe for use'''
df4 = pd.read_csv('2017_cases.csv').drop('Unnamed: 0',1)
df4

,search_terms,gtd_id,datetime,country,city,year
0,Cotabato City OR Maguindanao OR Philippines OR...,201712310032,2017-12-31,Philippines,Cotabato City,2017
1,Bangsamoro Islamic Freedom Movement (BIFM) OR ...,201712310016,2017-12-31,Philippines,Shariff Aguak,2017
2,Pikit OR North Cotabato OR Philippines OR,201712310011,2017-12-30,Philippines,Pikit district,2017
3,Bangsamoro Islamic Freedom Movement (BIFM) OR ...,201712310010,2017-12-31,Philippines,Limpogo,2017
4,Poblacion OR Cotabato City OR Maguindanao OR P...,201712300010,2017-12-30,Philippines,Cotabato City,2017
5,New People's Army (NPA) OR Menardo Cui OR Tuae...,201712290008,2017-12-28,Philippines,Near Tuael,2017
6,Bangsamoro Islamic Freedom Movement (BIFM) OR ...,201712280019,2017-12-28,Philippines,Shariff Aguak,2017
7,Bangsamoro Islamic Freedom Movement (BIFM) OR ...,201712270014,2017-12-28,Philippines,Datu Saudi-Ampatuan district,2017
8,Bangsamoro Islamic Freedom Movement (BIFM) OR ...,201712250017,2017-12-25,Philippines,Iginampong,2017
9,Total Access Communication Public Company Limi...,201712250006,2017-12-25,Thailand,Near Toteng,2017


In [4]:
main_df = pd.DataFrame()

#change the parameters here
timelapse  = 20
n1, n2 = 287,553
csvfile = 'nlinks3.csv'

pd.read_csv(csvfile)



'Error: 209. Need to train for cases where search terms did not match documents'

In [ ]:
'''Error: 209. Need to train for cases where search terms did not match documents'''

In [5]:
for i in range(n1,n2):
    while True:
        try:
            print(str(i) + ' out of ' + str(n2) + ' complete')
            keywords = 'false misleading allegation hoax rumour denied inaccurate dispute claimed'
            search_terms = df4['search_terms'][i] + ' ' + str(df4['year'][i]) + ' ' + keywords
            print(search_terms)
            
            csv_input = pd.read_csv(csvfile)
            df_lambda = pd.DataFrame(search_session(search_terms))
            df_lambda.columns = [df4['gtd_id'][i]]
            main_df = pd.concat([csv_input,df_lambda],axis = 1 )
            main_df.to_csv(csvfile, index=False)            
            
        #if run into capcha
        except ValueError as e:
            print(e)
            print('trying again after ' + str(timelapse) + ' mins')
            time.sleep(timelapse*60) #sleep for 10 mins
            timelapse += 1
            print(timelapse)
            continue
        else:
            break

287 out of 553 complete
Maute Group OR Marawi OR Lanao OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
Length mismatch: Expected axis has 0 elements, new values have 1 elements
trying again after 20 mins
21
287 out of 553 complete
Maute Group OR Marawi OR Lanao OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
Length mismatch: Expected axis has 0 elements, new values have 1 elements
trying again after 21 mins
22
287 out of 553 complete
Maute Group OR Marawi OR Lanao OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
Length mismatch: Expected axis has 0 elements, new values have 1 elements
trying again after 22 mins
23
287 out of 553 complete
Maute Group OR Marawi OR Lanao OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
288 out of 553 complete
Leovino Hidalgo OR Balete OR Poblacion OR Batangas O

316 out of 553 complete
Abu Sayyaf Group (ASG) OR Cathedral OR Lady Help OR Marawi OR Lanao OR Philippines OR Marawi Islamic OR Iraq OR Levant OR ISIL OR Sources OR Maute Group OR Bangsamoro Islamic Freedom Movement OR BIFM OR  OR Islamic State of Iraq and the Levant (ISIL) OR Cathedral OR Lady Help OR Marawi OR Lanao OR Philippines OR Marawi Sources OR Maute Group OR Abu Sayyaf Group OR ASG OR Bangsamoro Islamic Freedom Movement OR BIFM OR  OR Maute Group OR Cathedral OR Lady Help OR Marawi OR Lanao OR Philippines OR Marawi Islamic OR Iraq OR Levant OR ISIL OR Sources OR Abu Sayyaf Group OR ASG OR Bangsamoro Islamic Freedom Movement OR BIFM OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
317 out of 553 complete
Abu Sayyaf Group (ASG) OR Christian OR Marawi OR Lanao OR Philippines OR Marawi Islamic OR Iraq OR Levant OR ISIL OR Sources OR Maute Group OR Bangsamoro Islamic Freedom Movement OR BIFM OR  OR Islamic State of Iraq and the Levant (ISIL) OR Ch

339 out of 553 complete
New People's Army (NPA) OR Philippine National OR PNP OR Annafatan OR Cagayan OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
340 out of 553 complete
Abu Sayyaf Group (ASG) OR Public Works OR Highways OR DPWH OR Bangkal OR Sulu OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
341 out of 553 complete
New People's Army (NPA) OR Quirino OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
342 out of 553 complete
New People's Army (NPA) OR Tigatto OR Davao OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
343 out of 553 complete
New People's Army (NPA) OR Lorenzo Farm OR Pangyan OR Davao OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
344 out of 553 complete
New People's Army (NPA) OR Macondray Plastic OR Bunawan OR Davao OR Phi

377 out of 553 complete
Jamaah Ansharut Daulah OR Traffic OR Tuban OR East Java OR Indonesia OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
378 out of 553 complete
Electricity OR Thailand OR EGAT OR Kayomati OR Narathiwat OR Thailand Kayomati OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
379 out of 553 complete
Electricity OR Thailand OR EGAT OR Kayomati OR Narathiwat OR Thailand Kayomati OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
380 out of 553 complete
Barisan Revolusi Nasional (BRN) OR Electricity OR Thailand OR EGAT OR Kayu Khla OR Narathiwat OR Thailand Thailand OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
381 out of 553 complete
Barisan Revolusi Nasional (BRN) OR Electricity OR Thailand OR EGAT OR Mo Mawi OR Pattani OR Thailand Thailand OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
382 out of 55

411 out of 553 complete
Barisan Revolusi Nasional (BRN) OR Electricity OR Thailand OR EGAT OR Lubo Baya OR Narathiwat OR Thailand Thailand OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
412 out of 553 complete
Barisan Revolusi Nasional (BRN) OR Electricity OR Thailand OR EGAT OR Pase OR Narathiwat OR Thailand Thailand OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
413 out of 553 complete
Barisan Revolusi Nasional (BRN) OR Electricity OR Thailand OR EGAT OR Pase OR Narathiwat OR Thailand Thailand OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
414 out of 553 complete
Barisan Revolusi Nasional (BRN) OR Electricity OR Thailand OR EGAT OR Narathiwat OR Thailand Thailand OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
415 out of 553 complete
Barisan Revolusi Nasional (BRN) OR Electricity OR Thailand OR EGAT OR Narathiwat OR Thailand Thailand OR  2017 

Length mismatch: Expected axis has 0 elements, new values have 1 elements
trying again after 40 mins
41
448 out of 553 complete
Abu Sayyaf Group (ASG) OR Basakan OR Basilan OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
449 out of 553 complete
New People's Army (NPA) OR Jubgan OR Surigao OR Norte OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
450 out of 553 complete
New People's Army (NPA) OR Arman Matutina OR Buenavista OR Negros Occidental OR Philippines OR Matutina OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
451 out of 553 complete
New People's Army (NPA) OR Patpat OR Bukidnon OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
452 out of 553 complete
Abu Sayyaf Group (ASG) OR KMC Hardware OR Lamitan OR Basilan OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute

43
489 out of 553 complete
New People's Army (NPA) OR Lumanag Lipa OR Batangas OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
Length mismatch: Expected axis has 0 elements, new values have 1 elements
trying again after 43 mins
44
489 out of 553 complete
New People's Army (NPA) OR Lumanag Lipa OR Batangas OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
490 out of 553 complete
Jamaah Ansharut Daulah OR Pandawa Park OR Bandung OR West Java OR Indonesia OR Yayat Cahdiyat OR Additionally OR Cahdiyat OR Jamaah Ansharut Tauhid OR JAT OR  OR Jamaah Ansharut Tauhid (JAT) OR Pandawa Park OR Bandung OR West Java OR Indonesia OR Yayat Cahdiyat OR Jamaah Ansharut Daulah OR Cahdiyat OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
491 out of 553 complete
New People's Army (NPA) OR Merangerang OR Bukidnon OR Philippines OR  2017 false misleading allegation hoax rumour

527 out of 553 complete
New People's Army (NPA) OR Lebe OR Kiamba OR Sarangani OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
528 out of 553 complete
New People's Army (NPA) OR Benguet OR Mabbayad OR Echague OR Isabela OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
529 out of 553 complete
Abu Sayyaf Group (ASG) OR Danapah OR Basilan OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
530 out of 553 complete
New People's Army (NPA) OR Pico OR Loro Resort OR Payapa OR Batangas OR Philippines OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
531 out of 553 complete
Ko Ni OR Muslim OR National League OR Democracy Party OR Yangon Airport OR Yangon OR Myanmar OR Kyi Linn OR No OR  2017 false misleading allegation hoax rumour denied inaccurate dispute claimed
532 out of 553 complete
Benito Flores Clamosa OR Daang A

In [12]:
main_df.drop('a',1,inplace=True)

Unpivot the dataframe

In [13]:
df = main_df.unstack().reset_index().dropna()

In [14]:
df.drop('level_1',1,inplace=True)
df.columns = ['gtd_id','link']
df

,gtd_id,link
0,201712310032,http://www.cnn.com/2010/TECH/01/14/twitter.hoa...
1,201712310032,https://www.npr.org/2017/10/29/560569373/are-s...
2,201712310032,https://www.quora.com/What-is-the-difference-b...
3,201712310032,http://www.courtisane.be/sites/default/files/l...
4,201712310032,http://www.protectingeducation.org/sites/defau...
5,201712310032,http://www.tandfonline.com/doi/pdf/10.1080/045...
6,201712310032,http://eresources.nlb.gov.sg/newspapers/digiti...
7,201712310032,http://eresources.nlb.gov.sg/newspapers/digiti...
8,201712310032,http://www.forestpeoples.org/sites/fpp/files/p...
9,201712310032,https://adst.org/wp-content/uploads/2018/04/Ph...


In [15]:
df.to_csv('2017links.csv')

In [ ]:
df = pd.read_excel('nlinks2.xlsx')